# **Instruções Gerais**

- Copie este notebook, junto com as imagens da pasta para o seu Google Drive da UFV
- Resolva o que está sendo pedido no próprio notebook
- Lembre-se de montar o seu Google Drive no sistema de arquivos do notebook (ícone da pastinha, na barra de ferramentas à esquerda)
- Entregue, como resposta à atividade no **Moodle**

- **Obs.:**
  - Lembre-se de **salvar** o notebook de tempos em tempos
  - Envie **apenas** o notebook. Não há necessidade de enviar as imagens

# **1. Preâmbulo**

## **1.1. Identifique-se**

Execute a célula a seguir para se identificar

In [ ]:
estudante = input('Informe seu nome completo: ')
matricula = int(input('Informe sua matrícula (apenas números): '))

print('\n\nOlá %s (%d)!' % (estudante, matricula))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## **1.2. Importações de módulos Python**

- As bibliotecas necessárias para a atividade já se encontram importadas na célula a seguir. Mas você pode adicionar novas importações, se achar necessário

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## **1.3. Funções auxiliares já implementadas**

- Para informações mais detalhadas a respeito das funções, consulte o notebook de aula do capítulo "Entropia, Ruídos e Métricas"

### **1.3.1. Função `exibe_mosaico`**

In [ ]:
def exibe_mosaico(imagens, legendas, colunas=4, arquivo=None):
  # Largura e altura básicos, de referência, das imagens
  l = 8
  a = 6

  n_imagens = len(imagens)
  if len(imagens) < colunas:
    colunas = len(imagens)
  linhas = n_imagens // colunas + int(n_imagens % colunas > 0)

  fig = plt.figure(figsize=(l*colunas,a*linhas))
  ax = []

  i = j = 0
  for pos, img in enumerate(imagens):
    i = pos // colunas
    j = pos % colunas
    ax.append(fig.add_subplot(linhas, colunas, pos+1))
    ax[-1].axis('off')
    if img.mode == 'L' or img.mode == '1':
      img = img.convert('RGB')
    arr = np.asarray(img)
    ax[-1].imshow(arr)
    ax[-1].set_title(legendas[pos])

  fig.tight_layout()

  if arquivo is not None:
    plt.savefig(arquivo, dpi=300)

  plt.show()

### **1.3.2. Função `gera_imagem`**

In [ ]:
def gera_imagem(arr_orig):
  arr = arr_orig.copy()
  arr[arr > 255] = 255
  arr[arr < 0] = 0
  arr = np.round(arr).astype(np.uint8)
  return Image.fromarray(arr)

### **1.3.3. Função `cinza_lum`**

- Produz uma imagem monocromática, em tons de cinza, usando a técnica conhecida como *luminosity*
- Parâmetros:
  - `img`: imagem PIL, em RGB, para a qual se deseja converter para tons de cinza
  - `rgb`: *flag* para indicar se a imagem resultante deverá ser uma monocromática "verdadeira" (matriz bidimensional) ou monocromática RGB, usando 3 canais, porém todos com o mesmo valor (matriz tridimensional). Opcional. Valor *default*: `False`
  - `ret_img`: *flag* para indicar se o retorno deve ser um objeto *PIL Image* (valor verdadeiro) ou *array numpy* (valor falso). Opcional. Valor *default*: `False`
- Retorno:
  - Objeto *PIL Image* ou *array numpy*, de acordo com a especificação dos parâmetros

In [ ]:
def cinza_lum(img, rgb=False, ret_img=False):
  arr = np.asarray(img).astype(float)

  T = [0.299, 0.587, 0.114]

  T = np.array(T).T

  arrC = np.dot(arr, T)

  if rgb:
    arrC = np.dstack((arrC, arrC, arrC))

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC

## **1.4. Inicialização de *path***

- Lembre-se de alterar o valor da string de path para a pasta do seu Drive onde você salvou o notebook e as imagens.
- O caminho da string deve terminar com um caractere de barra (`/`), para que o restante do código funcione sem alterações.

In [ ]:
path = '/content/drive/MyDrive/Praticas PDI/'

# **2. Exercícios**

## **2.1. Geração de imagem monocromática: tons de cinza**

Pode-se produzir imagens em tons de cinza de várias maneiras. Iremos aqui implementar algumas dessas soluções e também treinar diferentes formas de tratar o armazenamento dos dados.

- Já vimos, em sala, a transformação usando a técnica conhecida como *luminosity*, que já se encontra implementada nas funções auxiliares, acima.

#### **Preparando os testes**

Execute a célula abaixo para visualizar a imagem original e sua versão cinza *luminosity*.


In [ ]:
img = Image.open(path + 'blue_tang.jpg')
imgLum = cinza_lum(img, ret_img=True)

imagens = [img, imgLum]
legendas = ['Original', 'Luminosity']

exibe_mosaico(imagens, legendas)

### **2.1.1. Decomposição de máximo e decomposição de mínimo**

Estas técnicas são análogas entre si e consistem em se produzir uma imagem em tons de cinza utilizando o maior dos valores entre R, G e B de cada pixel, para a decomposição de máximo, e o menor destes mesmos valores, para a decomposição de mínimo.

Implemente os corpos da funções `decomp_max` e `decomp_min`, a seguir, para que produzam os resultados descritos. As assinaturas das funções devem ser a mesma da função `cinza_lum` que se encontra pronta (exceto pelo nome, obviamente), bem como as possibilidades de retorno.

Você pode aproveitar a estrutura geral da função pronta para gerar as suas.

In [ ]:
np.amax(np.asarray(img).astype(float))

In [ ]:
def decomp_max(img, rgb=False, ret_img=False):
  arr = np.asarray(img).astype(float)

  h, w, _ = arr.shape

  arrC = np.zeros((h, w))

  for i in range(h):
    for j in range(w):
      arrC[i, j] = np.max(arr[i, j])

  if rgb:
    arrC = np.dstack((arrC, arrC, arrC))

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC

#### **Testando a implementação**

In [ ]:
imgDmax = decomp_max(img, ret_img=True)

imagens.append(imgDmax)
legendas.append('Decomposição de Máximo')

exibe_mosaico(imagens, legendas)

In [ ]:
def decomp_min(img, rgb=False, ret_img=False):
  arr = np.asarray(img).astype(float)

  h, w, _ = arr.shape

  arrC = np.zeros((h, w))

  for i in range(h):
    for j in range(w):
      arrC[i, j] = np.min(arr[i, j])

  if rgb:
    arrC = np.dstack((arrC, arrC, arrC))

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC

#### **Testando a implementação**

In [ ]:
imgDmin = decomp_min(img, ret_img=True)

imagens.append(imgDmin)
legendas.append('Decomposição de Mínimo')

exibe_mosaico(imagens, legendas)

### **2.1.2. Média**

Esta técnica consiste em se produzir uma imagem em tons de cinza utilizando a média dos valores R, G e B de cada pixel.

Implemente o corpo da função `cinza_media`, a seguir, para que produza o resultado descrito. Como no exercício anterior, siga a estrutura geral da função `cinza_lum`.

In [ ]:
def cinza_media(img, rgb=False, ret_img=False):
  arr = np.asarray(img).astype(float)

  h, w, _ = arr.shape

  arrC = np.zeros((h, w))

  for i in range(h):
    for j in range(w):
      arrC[i, j] = np.mean(arr[i, j])

  if rgb:
    arrC = np.dstack((arrC, arrC, arrC))

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC

#### **Testando a implementação**

In [ ]:
imgMed = cinza_media(img, ret_img=True)

imagens.append(imgMed)
legendas.append('Média')

exibe_mosaico(imagens, legendas)

### **2.1.3. *Lightness***

Esta técnica consiste em se produzir uma imagem em tons de cinza a partir dos valores R, G e B de cada pixel, segundo a regra abaixo:

$p' = \frac{max(R,G,B) + min(R,G,B)}{2}$

Implemente o corpo da função `cinza_lightness`, a seguir, para que produza o resultado descrito. Como nos exercícios anteriores, siga a estrutura geral da função `cinza_lum`.

In [ ]:
def cinza_lightness(img, rgb=False, ret_img=False):
  arr = np.asarray(img).astype(float)

  h, w, _ = arr.shape

  arrC = np.zeros((h, w))

  for i in range(h):
    for j in range(w):
      arrC[i, j] = (np.max(arr[i, j]) + np.min(arr[i, j]))/2

  if rgb:
    arrC = np.dstack((arrC, arrC, arrC))

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC

#### **Testando a implementação**

In [ ]:
imgLight = cinza_lightness(img, ret_img=True)

imagens.append(imgLight)
legendas.append('Lightness')

exibe_mosaico(imagens, legendas)

### **2.1.4. Informação de um único canal**

Esta técnica consiste em se produzir uma imagem em tons de cinza escolhendo a informação de um único canal, entre R, G e B.

Implemente o corpo da função `cinza_canal`, a seguir, para que produza o resultado descrito. Como nos exercícios anteriores, siga a estrutura geral da função `cinza_lum`, porém, desta vez, com um parâmetro adicional, chamado `canal`, que informa qual canal será utilizado para gerar a imagem cinza. O parâmetro será uma string, identificando o canal em questão ('r', 'g' ou 'b').

In [ ]:
CANAL_MAPA = {
  'R': 0,
  'G': 1,
  'B': 2
}

def cinza_canal(img, canal, rgb=False, ret_img=False):
  c_escolhido = CANAL_MAPA.get(canal, 0)

  arr = np.asarray(img).astype(float)

  h, w, _ = arr.shape

  arrC = np.zeros((h, w))

  for i in range(h):
    for j in range(w):
      arrC[i, j] = arr[i, j][c_escolhido]

  if rgb:
    arrC = np.dstack((arrC, arrC, arrC))

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC



#### **Testando a implementação**

In [ ]:
canais = ['R', 'G', 'B']

for canal in canais:
  imgCanal = cinza_canal(img, canal, ret_img=True)

  imagens.append(imgCanal)
  legendas.append('Apenas Canal %s' % canal)

exibe_mosaico(imagens, legendas)

### **2.1.5. EXERCÍCIO BÔNUS: Comparando métodos**

Este exercício vale **pontos extras** para quem o cumprir corretamente. Portanto, **não é obrigatório** fazê-lo e, caso deixe sua solução em branco, isso não afetará negativamente sua nota.

Escreva, na célula a seguir, um código que compara os resultados de cada uma das suas implementações acima com um resultado de referência, produzido pela função `cinza_lum`. O programa deve "printar" o grau de diferença entre cada imagem e a imagem de referência e por fim, informar qual foi a mais parecida.

Para este exercício, pede-se que se realize os testes usando a imagem `fitas_bonfim.jpg`, cujas diferenças de resultados são mais sutis visualmente do que a imagem do peixe.

In [ ]:
def root_mse(arr, other):
  return np.sqrt(np.sum((arr-other)**2))

In [ ]:
################# COMPLETE COM SEU CÓDIGO #####################
imagens_np = list(map(lambda im: np.asarray(im), imagens))

In [ ]:
for i in range(1, len(imagens_np)):
  print(f'Root Mean Squared Error: {root_mse(imagens_np[0], imagens_np[i])}')

## **2.2. Geração de imagem monocromática com paleta da cor sépia**

Este resultado pode ser obtido de forma semelhante à que fizemos com a técnica da luminância. Porém, lá estávamos interessados em uma matriz bidimensional, pois isso basta para a representação dos tons de cinza.

Aqui, no entanto, precisamos de uma matriz tridimensional, pois a base da coloração (sépia) não é um tom de cinza. Desta forma, nosso resultado final tem, também, que ser uma matriz RGB.

Se, para a imagem monocromática em cinza, nós utilizamos um produto matricial de uma matriz $m \times n \times 3$ por uma outra $3 \times 1$, obtendo uma matriz resultante com dimensões $m \times n$, aqui precisamos que o resultado também seja uma matriz $m \times n \times 3$, como a original.

Assim, precisamos de uma matriz de transformação não mais $3 \times 1$, mas uma $3 \times 3$. Para isto, precisamos de uma matriz quadrada que tenha alguma distribuição de pesos para que obtenhamos o resultado esperado.

No caso dos tons de sépia, esta matriz é:

$\begin{pmatrix}0.393 & 0.349 & 0.272\\ 0.769 & 0.686 & 0.534 \\ 0.189 & 0.168 & 0.131\end{pmatrix} $

Preencha o corpo da função `mono_sepia`, de forma que a mesma produza uma versão monocromática, em tons de sépia, da imagem original. A estrutura da função deve seguir análoga às dos exercícios anteriores, execeto pelo fato de que, desta vez, não há o parâmetro da opção de retornar um resultado RGB ou não, por que o resultado necessariamente estará em RGB.

In [ ]:
def mono_sepia(img, ret_img=False):
  arr = np.asarray(img).astype(float)

  M_TRANS = [[.393, .349, .272],
            [.769, .686, .534],
            [.189, .168, .131]]

  arrC = np.dot(arr, M_TRANS)

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC

#### **Testando seu código**

In [ ]:
img = Image.open(path + 'blue_tang.jpg')
img2 = Image.open(path + 'fitas_bonfim.jpg')

sepia = mono_sepia(img, ret_img=True)
sepia2 = mono_sepia(img2, ret_img=True)

imagens = [img, sepia, img2, sepia2]
legendas = ['Original', 'Versão em Sépia', 'Original', 'Versão em Sépia']

exibe_mosaico(imagens, legendas)

## **2.3 Binarização simples de imagens**

Uma maneira bem simples e direta de se produzir uma imagem binária (em preto e branco) é utilizar um parâmetro de limiar (*threshold*) e uma versão monocromática (em cinza) da imagem original.

Implemente o corpo da função `binariza`, que recebe como parâmetros a imagem original, o valor (inteiro) do limiar `th` e a *flag* de controle do retorno do array de dados ou do objeto da imagem.

Siga os seguintes passos, internamente à função:

- Converta a imagem original para tons de cinza, usando *luminosity*
- Nos pixels cujo valor de cinza é maior que `th`, transforme o resultado para branco (255)
- Nos pixels cujo valor vai até `th`, transforme o resultado para preto (0)
- Retorne o resultado:
  - Se pedido o array, retorne um array bidimensional
  - Se pedida a imagem, retorne uma imagem monocromática (em termos de estrutura de dados, porém binária em termos dos valores presentes)

In [ ]:
def binariza(img, th, ret_img=False):
  arr = cinza_lum(img, False)
  arr[arr > th] = 255
  arr[arr <= th] = 0

  return gera_imagem(np.dstack((arr, arr, arr))) if ret_img else arr


#### **Testando a implementação**

In [ ]:
imagens = [img]
legendas = ['Original']

limiares = [25, 50, 75, 100, 127, 200, 225]

for th in limiares:
  imgBin = binariza(img, th, ret_img=True)
  imagens.append(imgBin)
  legendas.append('Binarização com limiar = %d' % th)

exibe_mosaico(imagens, legendas)

## **2.4 Ajuste de brilho e contraste**

Implemente o corpo da função `brilho_contraste`, que, como o nome sugere, deve ajustar o brilho e o contraste da imagem de entrada.

Parâmetros:
- `c`: fator de contraste. Um ponto flutante contido no intervalo $[0.1,2]$. Opcional. Valor *default*: $1$.
- `b`: fator de brilho. Também um ponto flutuante, porém contido no intervalo $[-1,1]$. Opcional. Valor *default*: $0$
- `ret_img`: mesma interpretação e especificação dos exercícios anteriores

**Obs.:** o fator de brilho `b` não deve ser utilizado diretamente para produzir o brilho na imagem, ele deve ser utilizado para que se obtenha um **inteiro**, no intervalo $[0,255]$, este sim, o fator de brilho aplicado na imagem.

**FEATURE BÔNUS:** ganhe pontos extras se implementar o lançamento de exceções para quando os parâmetros `c` e `b` extrapolam os limites dos intervalos pedidos. Feature opcional.

In [ ]:
def brilho_contraste(img, c=1, b=0, ret_img=False):
  if c <= 0 or c > 2: raise Exception('Erro: Contraste fora do limite [.1, 2]!')
  if b < -1 or b > 1: raise Exception('Erro: Brilho fora do limite [-1, 1]!')

  arr = np.asarray(img).astype(float)
  f_brilho = b * 255

  arr /= c
  arr += f_brilho

  if ret_img:
    return gera_imagem(arr)
  else:
    return arr

#### **Testando sua implementação**

In [ ]:
img = Image.open(path + 'fitas_bonfim.jpg')
imagens = [img]
legendas = ['Original']

cont = [1, 0.5, 1.5]
brilho = [-0.2, 0, 0.5]

for c in cont:
  for b in brilho:
    imgAjustada = brilho_contraste(img, c, b, ret_img=True)
    imagens.append(imgAjustada)
    legendas.append('Ajustada com c=%.1f e b=%.1f' % (c,b))

exibe_mosaico(imagens, legendas)

## **2.5 Mudança de modelos de cor**

Complete o código abaixo (dentro do trecho delimitado por comentários) para implementar a conversão de uma imagem RGB em uma imagem CMYK, para posterior visualização das informações de cada canal. Produza arrays numpy com os nomes `C`, `M`, `Y` e `K` (em maiúsculas mesmo). O ajuste para visualização dos canais já se encontra implementado, portanto, não se preocupe com este aspecto.

In [ ]:
img = Image.open(path + 'blue_tang.jpg')
arr = np.asarray(img).astype(float)

imagens = [img]
legendas = ['Original']

############# INÍCIO DO TRECHO EDITÁVEL #############

rL = arr[:,:,0]/255
gL = arr[:,:,1]/255
bL = arr[:,:,2]/255

print(rL.shape)

K = np.zeros((arr.shape[0], arr.shape[1]))

for i in range(arr.shape[0]):
  for j in range(arr.shape[1]):
    K[i,j] = 1 - np.max((rL[i,j], gL[i,j], bL[i,j]))

C = (1 - rL - K)/(1 - K)

M = (1 - gL - K)/(1 - K)

Y = (1 - bL - K)/(1 - K)

#############  FIM DO TRECHO EDITÁVEL   #############

# Ajuste de visualização
c = C * 255
imgC = gera_imagem(c)
imagens.append(imgC)
legendas.append('Canal C')

m = M * 255
imgM = gera_imagem(m)
imagens.append(imgM)
legendas.append('Canal M')

y = Y * 255
imgY = gera_imagem(y)
imagens.append(imgY)
legendas.append('Canal Y')

k = K * 255
imgK = gera_imagem(k)
imagens.append(imgK)
legendas.append('Canal K')

exibe_mosaico(imagens, legendas)

#### **Continuação**

Complete, agora, o código a seguir para que a imagem RGB original seja recomposta a partir das matrizes `C`, `M`, `Y` e `K`, do exemplo anterior. Salve o resultado no array `arrReconstr`.

Para se ter certeza de que a reconstrução foi bem sucedida, o resultado da métrica RMSE deve ser zero.

In [ ]:
def rmse(arr1, arr2):
  return np.sqrt(np.sum((arr1-arr2)**2))

############# INÍCIO DO TRECHO EDITÁVEL #############

R = 255 * (1-C)*(1-K)
G = 255 * (1-M)*(1-K)
B = 255 * (1-Y)*(1-K)

arrReconstr = np.dstack((R, G, B))

#############  FIM DO TRECHO EDITÁVEL   #############

print('RMSE = %.3f' % rmse(arr, arrReconstr))

#### **Continuação**

Implemente o corpo de uma função para clarear/escurecer uma imagem usando o espaço de cores CMYK, conforme mostrado nos slides de 65 a 67 do capítulo de "Operações Ponto a Ponto"

A função deve se chamar `controle_preto` e ter como parâmetros a imagem original, o fator de ajuste no canal do preto e a *flag* de retorno da imagem ou array.

In [ ]:
def controle_preto(img, fator, ret_img=False):
  arr = np.asarray(img)
  rL = arr[:,:,0]/255
  gL = arr[:,:,1]/255
  bL = arr[:,:,2]/255

  print(rL.shape)

  K = np.zeros((arr.shape[0], arr.shape[1]))

  for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
      K[i,j] = 1 - np.max((rL[i,j], gL[i,j], bL[i,j]))

  C = (1 - rL - K)/(1 - K)
  M = (1 - gL - K)/(1 - K)
  Y = (1 - bL - K)/(1 - K)

  K *= fator

  R = 255 * (1-C)*(1-K)
  G = 255 * (1-M)*(1-K)
  B = 255 * (1-Y)*(1-K)

  arrC = np.dstack((R, G, B))

  if ret_img:
    return gera_imagem(arrC)
  else:
    return arrC


#### **Testando seu código**

In [ ]:
img = Image.open(path + 'blue_tang.jpg')

imagens = [img]
legendas = ['Original']

fatores = [0.2, 0.8, 1.2, 1.8]
for fator in fatores:
  imgPreto = controle_preto(img, fator, ret_img=True)
  imagens.append(imgPreto)
  legendas.append('Fator de ajuste = %.1f' % fator)

exibe_mosaico(imagens, legendas)

#### **Bons estudos!**

- Lembre-se de enviar apenas o notebook